# NN by gaurav

### Load libraries and data

In [1]:
import pandas as pd
import numpy as np
# import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [26]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
# check missing values per column
train.isnull().sum(axis=0)/train.shape[0]

ID             0.000000
datetime       0.000000
siteid         0.099896
offerid        0.000000
category       0.000000
merchant       0.000000
countrycode    0.000000
browserid      0.050118
devid          0.149969
click          0.000000
dtype: float64

In [4]:
train.click.value_counts()/train.shape[0]

0    0.963979
1    0.036021
Name: click, dtype: float64

In [27]:
df_ones = train[train.click==1]
df_zero = train[train.click==0]

In [110]:
rows = np.random.choice(df_zero.index.values, 430000)
print(df_ones.shape[0])
new_df = df_ones.append(df_zero.loc[rows])
print(new_df.shape[0])
rows = np.random.choice(new_df.index.values, new_df.shape[0])
new_df = new_df.loc[rows]

437214
867214


### Clean Data and Create Features

In [111]:
new_df['siteid'].fillna(-999, inplace=True)
test['siteid'].fillna(-999, inplace=True)

new_df['browserid'].fillna("None", inplace=True)
test['browserid'].fillna("None", inplace=True)

new_df['devid'].fillna("None", inplace=True)
test['devid'].fillna("None", inplace=True)

In [112]:
# set datatime
new_df['datetime'] = pd.to_datetime(new_df['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

In [113]:
# create datetime variable
new_df['tweekday'] = new_df['datetime'].dt.weekday
new_df['thour'] = new_df['datetime'].dt.hour
new_df['tminute'] = new_df['datetime'].dt.minute

test['tweekday'] = test['datetime'].dt.weekday
test['thour'] = test['datetime'].dt.hour
test['tminute'] = test['datetime'].dt.minute

In [114]:
cols = ['siteid','offerid','category','merchant']

for x in cols:
    new_df[x] = new_df[x].astype('object')
    test[x] = test[x].astype('object')

In [115]:
cat_cols = cols + ['countrycode','browserid','devid']

In [116]:
for col in cat_cols:
    lbl = LabelEncoder()
    lbl.fit(list(new_df[col].values) + list(test[col].values))
    new_df[col] = lbl.transform(list(new_df[col].values))
    test[col] = lbl.transform(list(test[col].values))

In [124]:
train.head()

[1 0 1 ..., 0 1 1]


### Model Training

In [118]:
cols_to_use = list(set(train.columns) - set(['ID','datetime','click']))

In [139]:
X_train, X_test, y_train, y_test = train_test_split(train[cols_to_use], train['click'], test_size = 0.5)
# print(y_train)

In [136]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
import time

model = MLPClassifier(hidden_layer_sizes=(10,1000,1000,1000,10), verbose=True)#, batch_size=X_train.shape[0])
start = time.time()
model.fit(X_train, y_train)
print("time to fit: ", time.time()-start)
start = time.time()
pred = model.predict(X_test)
print("time to predict: ", time.time()-start)
print("training score: ", roc_auc_score(y_train, model.predict(X_train)))
print("crossval score: ", roc_auc_score(y_test, pred))

Iteration 1, loss = 0.73022699
Iteration 2, loss = 0.69334152
Iteration 3, loss = 0.69334567
Iteration 4, loss = 0.69333637
Iteration 5, loss = 0.69334147
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
time to fit:  1543.3896703720093
time to predict:  19.673012018203735
training score:  0.5
crossval score:  0.5


In [ ]:
from sklearn.metrics import accuracy_score
print("acc. training score: ", accuracy_score(y_train, model.predict(X_train)))
print("acc. crossval score: ", accuracy_score(y_test, pred))

In [130]:
submit = pd.DataFrame(model.predict_proba(test[cols_to_use]))[1]

In [ ]:
pred = pd.DataFrame(pred)
print(pred[0].value_counts())
y_test = pd.Series(y_test)
print(y_test.value_counts())

After getting no results from NN, Checking a DTC.

In [140]:
from sklearn.tree import  DecisionTreeClassifier
import time

model =DecisionTreeClassifier
start = time.time()
model.fit(X_train, y_train)
print("time to fit: ", time.time()-start)
start = time.time()
pred = model.predict(X_test)
print("time to predict: ", time.time()-start)
print("training score: ", roc_auc_score(y_train, model.predict(X_train)))
print("crossval score: ", roc_auc_score(y_test, pred))

ValueError: could not convert string to float: 'Edge'

In [30]:
sub = pd.DataFrame({'ID':test['ID'], 'click':submit})
sub.to_csv('nn_1.csv', index=False)